In [1]:
using Pkg
Pkg.instantiate()
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Distributions")
Pkg.add("Unitful")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [1]:
using CSV
using DataFrames
using Random, Distributions
using Unitful

In [22]:
include("src/main.jl")
include("src/temporal_buffer.jl")
include("test.jl")

pop_post.v = TemporalBuffer{Float32}(0.001, Float32[0.0 0.0; 0.0 0.0; 0.0 0.0])
(net.pops["2"]).v = TemporalBuffer{Float32}(0.001, Float32[0.0 0.0; 0.0 0.0; 0.0 0.0])


TemporalBuffer{Float32}(0.001, Float32[0.0 0.0; 0.0 0.0; 0.0 0.0])

In [13]:
net = NeuronNet()

NeuronNet(Dict{String, NeuronPopulation}(), Dict{Tuple{String, String}, AbstractArray}())

In [14]:
l_name = ["2/3E", "2/3I", "4E", "4I", "5E", "5I", "6E", "6I"]
x = [1148, 324, 268, 60, 1216, 304, 800, 164, 657]
n_layer = [i * 9 for i in x]
n_layer_dict = Dict(zip(l_name, n_layer))

Dict{String, Int64} with 8 entries:
  "5I"   => 2736
  "4I"   => 540
  "6I"   => 1476
  "2/3I" => 2916
  "4E"   => 2412
  "2/3E" => 10332
  "5E"   => 10944
  "6E"   => 7200

In [15]:
df = DataFrame(CSV.File("shimoura11_spatial.csv"))

Row,Source,SourceType,Target,TargetType,Pmax,Radius,Weight,Wstd,Delay,Dstd
,String3,String1,String3,String1,Float64,Float64,String15,String7,Float64,Float64
1,2/3,E,2/3,E,0.192,0.0003,87.8*pA,8.78*pA,1.5,0.75
2,2/3,I,2/3,E,0.3095,0.000175,-351.2*pA,8.78*pA,0.8,0.4
3,4,E,2/3,E,0.3356,5.0e-5,175.6*pA,8.78*pA,1.5,0.75
4,4,I,2/3,E,0.5802,5.0e-5,-351.2*pA,8.78*pA,0.8,0.4
5,5,E,2/3,E,0.0143,5.0e-5,87.8*pA,8.78*pA,1.5,0.75
6,6,E,2/3,E,0.0159,5.0e-5,87.8*pA,8.78*pA,1.5,0.75
7,2/3,E,2/3,I,0.252,0.0003,87.8*pA,8.78*pA,1.5,0.75
8,2/3,I,2/3,I,0.2553,0.000175,-351.2*pA,8.78*pA,0.8,0.4
9,4,E,2/3,I,0.2558,5.0e-5,87.8*pA,8.78*pA,1.5,0.75


In [16]:
### Create synapse weight matrices
pA = 1
num_conn = size(df)[1]
for row_n in 1:num_conn
    src = df[row_n,:].Source*df[row_n,:].SourceType
    tgt = df[row_n,:].Target*df[row_n,:].TargetType
    #println(src, " to ", tgt)
    prob = df[row_n,:].Pmax
    nsyn = round(Int, log(1.0-prob)/log(1.0 - (1.0/(n_layer_dict[src]*n_layer_dict[tgt]))))
    #println("Num Syn:", nsyn)
    src_idx = rand(1:n_layer_dict[src], nsyn)
    tgt_idx = rand(1:n_layer_dict[tgt], nsyn)
    #pA = u"pA"
    Weight = eval(Meta.parse(df[row_n,:].Weight))
    Wstd = eval(Meta.parse(df[row_n,:].Wstd))
    d = Normal(ustrip(Weight), ustrip(Wstd))
    weights=rand(d,nsyn) #* u"pA"
    submatrix = Array{Float32, 2}(undef, n_layer_dict[src], n_layer_dict[tgt])
    for i in 1:nsyn
        submatrix[src_idx[i], tgt_idx[i]] = weights[i]
    end
    net.weights[(src, tgt)] = submatrix 
end

In [17]:
net.weights[("4E", "5E")]

2412×10944 Matrix{Float32}:
  0.0       0.0     96.2183  93.2749  79.5111  …    0.0     86.1519    0.0
  0.0       0.0     85.1694   0.0     79.9724       0.0      0.0      86.4727
 97.3559   66.3702   0.0      0.0      0.0         83.316   85.5474    0.0
  0.0      93.34     0.0     83.2298   0.0          0.0     90.9884   90.9589
  0.0       0.0      0.0      0.0      0.0          0.0      0.0       0.0
  0.0      76.2491  84.118    0.0      0.0     …   90.737    0.0      92.9193
  0.0       0.0      0.0      0.0      0.0        104.803   95.893     0.0
  0.0       0.0     89.882   92.056    0.0          0.0      0.0       0.0
  0.0      89.3576   0.0      0.0      0.0          0.0      0.0       0.0
  0.0     101.814    0.0      0.0     83.2156       0.0     94.471    88.9236
 96.2138    0.0     94.7604  84.0146  86.8728  …   83.6754   0.0       0.0
 93.6692   86.5879  92.6513  78.1463   0.0          0.0     82.2768   86.6117
 74.0959   84.8575  91.1541   0.0      0.0          0.0  

In [18]:
d_ex = 1.5e-3     	# Excitatory delay
std_d_ex = 0.75e-3 	# Std. Excitatory delay
d_in = 0.80e-3      # Inhibitory delay
std_d_in = 0.4e-3  	# Std. Inhibitory delay
for (key, value) in n_layer_dict
    if occursin("E", key) == true
        d = d_ex
        d_std = std_d_ex
        else occursin("I", key) == true
        d = d_in
        d_std = std_d_in
    end
    a= Normal(d, d_std)
    delays=Float32.(rand(a,n_layer_dict[key]))
    net.pops[key] = NeuronPopulation(1e-4, abs.(delays))
end

In [19]:
net.pops

Dict{String, NeuronPopulation} with 8 entries:
  "5I"   => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …
  "4I"   => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …
  "6E"   => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …
  "2/3I" => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …
  "4E"   => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …
  "2/3E" => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …
  "6I"   => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …
  "5E"   => NeuronPopulation(TemporalBuffer{Float32}(0.0001, Float32[0.0 0.0 … …

In [20]:
net.weights[("6I","5I")]

LoadError: KeyError: key ("6I", "5I") not found

In [11]:
function sim_step!(pop::NeuronPopulation, v::Vector{Float32}, weights::AbstractArray)
  # update postsynaptic population given incident voltages

  # calculate v_out
  # v_out = your equation, including action potentials, ...
  pop.v.buf[:,1] += weights*v
end 

function activation!(pop::NeuronPopulation)
   for i = 1:length(pop.delays)
        if pop.v.buf[i,1] > firing_threshold
            pop.v[i] = reset_v
        end 
   end
end

LoadError: UndefVarError: NeuronPopulation not defined

In [23]:
firing_threshold = -55e-3
reset_v = -60e-3
sim_step!(net)

LoadError: DimensionMismatch: second dimension of A, 2736, does not match length of x, 540

In [28]:
net.pops["2/3E"].v.buf

10332×48 Matrix{Float32}:
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.06  NaN  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [29]:
net.pops["5I"].v[1, 1.0f0]

LoadError: BoundsError: attempt to access 2736×22 Matrix{Float32} at index [1, 10001]